In [1]:
# -*- coding: utf-8 -*-

# <strong> A  - Import des modules </strong> 

In [8]:
import warnings
warnings.filterwarnings('ignore')

#########################################

import numpy as np
import pandas as pd

#########################################

import argparse
import json
import requests

#########################################

#!pip install import_ipynb
import import_ipynb
from P6_00_YelpAPI_Kengni_zanguim_Brice_062022 import get_my_key


importing Jupyter notebook from P6_00_YelpAPI_Kengni_zanguim_Brice_062022.ipynb


# <strong> B -  Constantes necessaires à la recherche de restaurants via l'API </strong>

In [3]:
# La clé d'accès à l'API qui peut être générée sur :  https://www.yelp.com/developers/v3/manage_app
API_KEY =  get_my_key()

#  Point d'accès vers l'API 
def ENDPOINT ( n , id="" ) :
    if n == 1 :   # Recherche de restaurants/business
        return  "https://api.yelp.com/v3/businesses/search"
    elif n == 2 :   # acquisition d'avis clients
        try :
            return  f"https://api.yelp.com/v3/businesses/{id}/reviews"
        except :
            raise TypeError("Vous n'avez pas fournis l'ID du business")
            
HEADERS = {"Authorization":f"bearer {API_KEY}"}

PARAMETERS = {"term" : "food",
              "categories" :"restaurants",
              "location" :"Paris",
              "locale":"fr_FR" ,
              "limit" : 50 ,
              "offset": 400,
              "radius": 40000
              }

# <strong> C - Envoi de requêtes à l'API Yelp et récupération des identfiants de business </strong>

In [4]:
business_id = []
Business_count = 200
find_restaurants = True


if find_restaurants :
    diff, n0 , out= 0 , 0, 0
    while len (business_id ) < Business_count :
        reponse = requests.get(url=ENDPOINT(1) , params=PARAMETERS , headers = HEADERS).json()
        for i in range( len( reponse["businesses"] ) ) :
            business_id.append( reponse["businesses"][i]["id"] )
        business_id = list(np.unique( business_id ))
        diff = len(business_id) - n0
        n0 = len(business_id)
        if diff :
            out = 0
        else :
            out += 1

        if out == 3 : break
    

# <strong> D - Sauvegarde des ID de restaurant dans un fichier CSV </strong>

In [5]:
# Lecture du fichier préexistant
id_csv = pd.read_csv("restaurants_id.csv")
id_csv.drop( [i for i in id_csv.columns if "Unnamed" in i] , axis = 1, inplace=True)

# Complation des données
id_csv = pd.concat([pd.DataFrame({"restaurants_id":business_id}) , id_csv], ignore_index=True)

# Supression de duplicatats
id_csv.drop_duplicates(inplace=True, )

#Sauvegarde
id_csv.to_csv("restaurants_id.csv")

# <strong> E - Acquisition des avis clients et données relatives </strong>

In [6]:
PARAMETERS = { "locale":"fr_FR"  }
find_reviews = False

if find_reviews :
    reviews_df = pd.DataFrame()
    for id in id_csv["restaurants_id"] :
        reponse = requests.get( url = ENDPOINT(n=2 ,id=id) , params=PARAMETERS , headers = HEADERS ).json()
        reviews_df = pd.concat([pd.DataFrame(reponse["reviews"]), reviews_df], ignore_index=True)
        
    reviews_df = reviews_df[["id","text", "rating"]]
    reviews_df.to_csv("restaurants_reviews.csv")
else :
    reviews_df = pd.read_csv("restaurants_reviews.csv")
    reviews_df.drop( [i for i in reviews_df.columns if "Unnamed" in i] , axis = 1, inplace=True)

In [7]:
reviews_df

,id,text,rating
0,_aK8XjzpK2wO5V1aNO0Qyg,D'abord Le cadre est tip-top : rétro et super ...,4
1,OFCCLKXK96TkqLcUt1IU-g,Si j'habitais tout près du Parisien (comprenez...,4
2,ZDZ2O1-5M2yh-wud8GHZZw,À quoi reconnaitre un bistro parisien au XXIèm...,5
3,NFdFHh-h5mNlpaQjQFMKJA,Alors...j'ai un gros problème avec la crêperie...,3
4,KpaWoVGPzH3GLH6YN9OZMg,"Le daily syrien, c'est un mélange atypique ent...",4
...,...,...,...
925,H4-cbPVt8tOI0vdTlAwJ9A,New kid in the block !\nPrometteur le garçon :...,3
926,A_m67RroIhQxZ5Opb4zt6g,"Etant réputé bon pour leur pho, le Pho 14 à Ch...",2
927,6FNi5D0mtUY005RLWFWjdA,L'un de mes restaurants préférés à Paris ! Je ...,5
928,-a60A96QM9krus9vY2qtLg,"Réservation faite pour deux pour 19h, le resta...",3
